In [ ]:
import os
import math
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from skimage.restoration import denoise_tv_bregman
from use_gen_map import gen_map
import pickle
from collections import defaultdict
# Ensure the required package is installed
#get inputs
spec_file = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-PyMT-Spectra-TIC.csv" #add spectra file location
mass_file = r"D:\git_MALDI\MALDI\MassList-230615PyMT-Fat-CMC-DHB-Pos-1-PyMT.csv" #add mass list file location
spot_file = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-RegionSpots.csv" #add spot file location

#UNCOMMENT THIS IF YOU WANT TO EXTRACT m/z FROM THE SPECTRA FILE
#os.system("grep m/z " + spec_file + " > mz")
    
#print("m/z is written in a file mz.")

f = open(r"D:\git_MALDI\MALDI\my_own\mz", "r") #change this to the location of the mz file
mz = f.readline()
mz = mz.rstrip("\n")
mz = mz.split(";")
f.close() 

#os.system("rm mz")

index_dict = defaultdict(list)
for i in range(1, len(mz)):
    rounded = round(float(mz[i]), 2)
    index_dict[rounded].append(i)

In [ ]:
out_dir = r"D:\git_MALDI\MALDI\output_dir2" #change this to the location of the output directory
outputtype = "png"

#masses = pd.read_csv("masses_ind.csv")

maps_all = dict() 

for keys, value in index_dict.items():
    v95_max = 0
    added_map = np.zeros((1, 1))
    for index in value:
        map = gen_map(index, mz_data_filename = spec_file, region_spots_filename = spot_file, spectra_sep = ";", spot_sep = ";", space = 40)
        if added_map.size == 1:
            added_map = map
        else:
            added_map += map
        
    b = added_map.flatten()
    if sum(b > 0) < 1:
        v95_max = 0
    else:
        v95_max = np.percentile(b[b > -1], 95) 

    plt.figure(figsize=(6.4, 5.2))
    ax = sn.heatmap(added_map, vmax = v95_max, square = True, cmap = 'rainbow')
    plt.ylim(0, added_map.shape[0])
    title = "m/z: " +  str(keys)
    print(index, title)
    ax.set(xlabel = 'x (mm)', ylabel = 'y (mm)', title = title)
    plt.xticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.yticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.xticks(rotation=0)
    #plt.show()
    filename = out_dir + "/" + "mz" + str(keys) + "." + outputtype
    plt.savefig(filename)
    print("Saving " + filename)
    # the following three commands clear too many open figures
    plt.cla()
    plt.clf()
    plt.close("all")

     ## denoising
    # record no data region
    no_data = []
    for l2 in range(added_map.shape[0]):
        for n2 in range(added_map.shape[1]):
            if math.isnan(added_map[l2, n2]): 
                no_data.append((l2, n2))
                added_map[l2, n2] = 0
    plt.figure(figsize=(6.4, 5.2))
    after_noise = denoise_tv_bregman(added_map, weight=0.03)

    # restore no data region
    for l2, n2 in no_data:
        after_noise[l2, n2] = np.nan
    b = after_noise.flatten()
    if sum(b > 0) < 1:
        v95_max = 0
    else:
        v95_max = np.percentile(b[b > -1], 95)

    if v95_max > 0:
        maps_all[mz[index]] = after_noise

    ax = sn.heatmap(after_noise, vmax = v95_max, square = True, cmap = 'rainbow')
    plt.ylim(0, added_map.shape[0])
    title = "m/z: " +  str(round(mz[index], 4))
    #print(j, title)
    ax.set(xlabel = 'x (mm)', ylabel = 'y (mm)', title = title)
    plt.xticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.yticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.xticks(rotation=0)
    #plt.show()
    filename = out_dir + "/" + "mz" + str(keys) + "_denoised." + outputtype
    plt.savefig(filename)
    print("Saving " + filename)
    # the following three commands clear too many open figures
    plt.cla()
    plt.clf()
    plt.close("all")
    ##

with open(r'D:\git_MALDI\MALDI\my_own\saved_dictionary2.pkl', 'wb') as f: #change this to the location of the saved dictionary
    pickle.dump(maps_all, f)